# Full Code

In [38]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_digits
from sklearn.decomposition import NMF, PCA
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVC

X, y = load_digits(return_X_y=True)

pipe = Pipeline(
    [
        ("scaling", MinMaxScaler()),
        # the reduce_dim stage is populated by the param_grid
        ("reduce_dim", "passthrough"),
        ("classify", LinearSVC(dual=False, max_iter=10000)),
    ]
)

N_FEATURES_OPTIONS = [2, 4, 8]
C_OPTIONS = [1, 10, 100, 1000]
param_grid = [
    {
        "reduce_dim": [PCA(iterated_power=7), NMF(max_iter=1_000)],
        "reduce_dim__n_components": N_FEATURES_OPTIONS,
        "classify__C": C_OPTIONS,
    },
    {
        "reduce_dim": [SelectKBest(mutual_info_classif)],
        "reduce_dim__k": N_FEATURES_OPTIONS,
        "classify__C": C_OPTIONS,
    },
]
reducer_labels = ["PCA", "NMF", "KBest(mutual_info_classif)"]

grid = GridSearchCV(pipe, n_jobs=-1, param_grid=param_grid)
grid.fit(X, y)

GridSearchCV(estimator=Pipeline(steps=[('scaling', MinMaxScaler()),
                                       ('reduce_dim', 'passthrough'),
                                       ('classify',
                                        LinearSVC(dual=False,
                                                  max_iter=10000))]),
             n_jobs=-1,
             param_grid=[{'classify__C': [1, 10, 100, 1000],
                          'reduce_dim': [PCA(iterated_power=7),
                                         NMF(max_iter=1000)],
                          'reduce_dim__n_components': [2, 4, 8]},
                         {'classify__C': [1, 10, 100, 1000],
                          'reduce_dim': [SelectKBest(score_func=<function mutual_info_classif at 0x000002166195D3F0>)],
                          'reduce_dim__k': [2, 4, 8]}])

# Walkthrough

# Load the Dataset

In [39]:
X, y = load_digits(return_X_y=True)

- load_digits(return_X_y=True): Loads the digits dataset. return_X_y=True means it returns the data as two separate variables:
- X: The feature matrix (pixel values of the digit images).
- y: The target vector (the actual digit labels, 0-9).

# Create a Pipeline:



In [40]:
pipe = Pipeline(
    [
        ("scaling", MinMaxScaler()),
        ("reduce_dim", "passthrough"),
        ("classify", LinearSVC(dual=False, max_iter=10000)),
    ]
)

This code defines a machine learning pipeline in scikit-learn (sklearn). Pipelines are a powerful way to chain together multiple processing steps, ensuring the same transformations are applied during both training and prediction.

Here's a breakdown of the pipeline:

`Pipeline(...)`:  This creates the pipeline object that will hold and organize the steps.

The pipeline has three steps:

1.  **`("scaling", MinMaxScaler())`**:
    *   `"scaling"`: This is simply a name for this step in the pipeline.
    *   `MinMaxScaler()`: This scales the features to a specific range (usually 0 to 1). This is important for many algorithms that are sensitive to the scale of different features.

2.  **`("reduce_dim", "passthrough")`**:
    *   `"reduce_dim"`:  Another name for the step, indicating its purpose.
    *   `"passthrough"`: This is a placeholder! It means this step currently does nothing. Later, tools like `GridSearchCV` will replace `"passthrough"` with different dimensionality reduction methods (like PCA, NMF, or SelectKBest) to find the best one.

3.  **`("classify", LinearSVC(dual=False, max_iter=10000))`**:
    *   `"classify"`: The name of the final classification step.
    *   `LinearSVC(dual=False, max_iter=10000)`: This is a linear Support Vector Machine (SVM) classifier.
        *   `dual=False`:  This setting is preferred when you have more samples than features.
        *   `max_iter=10000`:  This sets the maximum number of iterations the algorithm will run to try and find a solution.

# Define Hyperparameter Options:

In [41]:
N_FEATURES_OPTIONS = [2, 4, 8]
C_OPTIONS = [1, 10, 100, 1000]

**`N_FEATURES_OPTIONS`**: A list of options for the number of components/features to keep after dimensionality reduction.

**`C_OPTIONS`**: A list of options for the regularization parameter `C` of the `LinearSVC` classifier. `C` controls the trade-off between maximizing the margin and minimizing the classification error.

# Define the Parameter Grid:



In [42]:
param_grid = [
    {
        "reduce_dim": [PCA(iterated_power=7), NMF(max_iter=1_000)],
        "reduce_dim__n_components": N_FEATURES_OPTIONS,
        "classify__C": C_OPTIONS,
    },
    {
        "reduce_dim": [SelectKBest(mutual_info_classif)],
        "reduce_dim__k": N_FEATURES_OPTIONS,
        "classify__C": C_OPTIONS,
    },
]

**`param_grid`**: This is a list of dictionaries. Each dictionary defines a set of hyperparameter combinations to try. This setup allows `GridSearchCV` to explore different dimensionality reduction methods and their associated hyperparameters.

*   **First dictionary (PCA and NMF):**
    *   `"reduce_dim"`: `[PCA(iterated_power=7), NMF(max_iter=1000)]`: `GridSearchCV` will try both `PCA` and `NMF` for dimensionality reduction.
        *   `PCA(iterated_power=7)` is configured to improve the accuracy of results.
        *   `NMF(max_iter=1000)` is configured to prevent non-convergence warnings.
    *   `"reduce_dim__n_components"`: `N_FEATURES_OPTIONS`: For both `PCA` and `NMF`, it will try setting the number of components to 2, 4, and 8. The `__` is used to access the hyperparameters of a specific step in the pipeline.
    *   `"classify__C"`: `C_OPTIONS`: It will try different values of `C` for the `LinearSVC` classifier (1, 10, 100, 1000).

*   **Second dictionary (SelectKBest):**
    *   `"reduce_dim"`: `[SelectKBest(mutual_info_classif)]`: `GridSearchCV` will use `SelectKBest` with `mutual_info_classif` as the scoring function. `SelectKBest` selects the top `k` features that are most informative for classification.
    *   `"reduce_dim__k"`: `N_FEATURES_OPTIONS`: It will try selecting the top 2, 4, and 8 features.
    *   `"classify__C"`: `C_OPTIONS`: Same as before, it will try different values of `C` for the classifier.

# Define Reducer Labels:



In [43]:
reducer_labels = ["PCA", "NMF", "KBest(mutual_info_classif)"]

**`reducer_labels`**: This list just provides human-readable names for the dimensionality reduction methods. It's used later for visualization purposes.

# Create and Run GridSearchCV:

In [44]:
grid = GridSearchCV(pipe, n_jobs=-1, param_grid=param_grid)
grid.fit(X, y)

GridSearchCV(estimator=Pipeline(steps=[('scaling', MinMaxScaler()),
                                       ('reduce_dim', 'passthrough'),
                                       ('classify',
                                        LinearSVC(dual=False,
                                                  max_iter=10000))]),
             n_jobs=-1,
             param_grid=[{'classify__C': [1, 10, 100, 1000],
                          'reduce_dim': [PCA(iterated_power=7),
                                         NMF(max_iter=1000)],
                          'reduce_dim__n_components': [2, 4, 8]},
                         {'classify__C': [1, 10, 100, 1000],
                          'reduce_dim': [SelectKBest(score_func=<function mutual_info_classif at 0x000002166195D3F0>)],
                          'reduce_dim__k': [2, 4, 8]}])

`grid = GridSearchCV(pipe, n_jobs=1, param_grid=param_grid)`:

*   `GridSearchCV(...)`: Creates a `GridSearchCV` object.
    *   `pipe`: The pipeline you created earlier is passed as the estimator. `GridSearchCV` will train and evaluate this pipeline with different hyperparameter combinations.
    *   `n_jobs=1`: This means use only 1 CPU core. You can change this to `-1` to use all available cores and speed up the process if you have a multi-core processor.
    *   `param_grid`: The parameter grid you defined to specify which hyperparameters to search.

`grid.fit(X, y)`: This is where the magic happens! `GridSearchCV` does the following:

1.  **Cross-Validation**: It splits the data into multiple folds (by default, 5-fold cross-validation).
2.  **Training and Evaluation**: For each combination of hyperparameters in `param_grid`, it trains the pipeline on some folds and evaluates its performance on the remaining fold.
3.  **Best Model Selection**: It keeps track of the hyperparameter combination that yields the best average performance across all folds (e.g., highest accuracy).
4.  **Refitting**: Finally, it refits the pipeline with the best hyperparameters found on the entire dataset (`X`, `y`).

In essence, the code automates the process of trying out different preprocessing and model configurations to find the one that works best for your data and task.

# Follow-up Question 1: Why Use "passthrough" in the Pipeline?

In [45]:
pipe = Pipeline(
    [
        ("scaling", MinMaxScaler()),
        ("reduce_dim", "passthrough"),
        ("classify", LinearSVC(dual=False, max_iter=10000)),
    ]
)

**Why did we passthrough "reduce_dim"; couldn't we just input the "reduce_dim" options from the `param_grid` variable here, or not?**

You're right to question the use of `"passthrough"` in the `reduce_dim` step. It might seem like we could directly put the dimensionality reduction methods there. However, there are a couple of key reasons why `"passthrough"` is used as a placeholder in this scenario:

**1. `GridSearchCV` Needs a Fixed Pipeline Structure:**

*   `GridSearchCV` works by trying out different combinations of hyperparameters within a predefined pipeline structure. The *structure* of the pipeline itself (i.e., the number and types of steps) needs to remain consistent across all iterations of the grid search.
*   If we were to directly insert the dimensionality reduction methods (PCA, NMF, SelectKBest) into the pipeline definition, we would be changing the pipeline's structure itself within the `param_grid`. `GridSearchCV` is not designed to handle changes to the fundamental structure of the pipeline during the search process.

**2. Flexibility to Include or Exclude Dimensionality Reduction:**

*   Using `"passthrough"` allows us to effectively turn the dimensionality reduction step "on" or "off" during the grid search.
    *   When `"passthrough"` is used, no dimensionality reduction is performed.
    *   When `GridSearchCV` replaces `"passthrough"` with PCA, NMF, or SelectKBest, the dimensionality reduction is active.
*   This gives us the option to see how the classifier performs *without* any dimensionality reduction as well, which can be a useful baseline comparison.

**3. Different Dimensionality Reducers Have Different Hyperparameters:**

*   PCA, NMF, and SelectKBest have different hyperparameters (e.g., `n_components` for PCA/NMF, `k` for SelectKBest).
*   If we embedded them directly into the pipeline, it would be difficult to specify different hyperparameter options for each method within `param_grid`. `GridSearchCV` would have a harder time figuring out which hyperparameter belongs to which method.
*   By using `"passthrough"` and letting `GridSearchCV` substitute the actual methods, we can cleanly associate the correct hyperparameters with each method in the `param_grid` (using `"reduce_dim__n_components"` or `"reduce_dim__k"`).

**In Summary:**

The `"passthrough"` placeholder acts as a switch to enable or disable the dimensionality reduction step while maintaining a fixed pipeline structure that `GridSearchCV` can work with. It also helps organize the hyperparameters associated with different dimensionality reduction methods in a clear and manageable way within the `param_grid`.

It's a bit like having a modular setup where you can easily swap out different components (dimensionality reduction methods) without having to rewire the entire system.

# Follow-up Question 2: Cross-Validation and Different param_grid Dictionaries

In [46]:
param_grid = [
    {
        "reduce_dim": [PCA(iterated_power=7), NMF(max_iter=1_000)],
        "reduce_dim__n_components": N_FEATURES_OPTIONS,
        "classify__C": C_OPTIONS,
    },
    {
        "reduce_dim": [SelectKBest(mutual_info_classif)],
        "reduce_dim__k": N_FEATURES_OPTIONS,
        "classify__C": C_OPTIONS,
    },
]

**Is the line `"reduce_dim": [PCA(iterated_power=7), NMF(max_iter=1_000)]` doing PCA and NMF for the same 5 folds of training, and afterwards select k best is doing another 5 folds? Or is it doing PCA in 5 of the folds, NMF in another 5 folds, and select k best in another 5 folds?**

That's a great question about how `GridSearchCV` handles the different dimensionality reduction methods within the cross-validation process.

Here's how it works:

1.  `GridSearchCV` treats each *combination* of hyperparameters in `param_grid` as an **independent experiment**.
2.  In your `param_grid`, you have two dictionaries:

In [47]:
# PCA and NMF
{
    "reduce_dim": [PCA(iterated_power=7), NMF(max_iter=1_000)],
    "reduce_dim__n_components": N_FEATURES_OPTIONS,
    "classify__C": C_OPTIONS,
}

{'reduce_dim': [PCA(iterated_power=7), NMF(max_iter=1000)],
 'reduce_dim__n_components': [2, 4, 8],
 'classify__C': [1, 10, 100, 1000]}

In [48]:
# SelectKBest
{
    "reduce_dim": [SelectKBest(mutual_info_classif)],
    "reduce_dim__k": N_FEATURES_OPTIONS,
    "classify__C": C_OPTIONS,
}

{'reduce_dim': [SelectKBest(score_func=<function mutual_info_classif at 0x000002166195D3F0>)],
 'reduce_dim__k': [2, 4, 8],
 'classify__C': [1, 10, 100, 1000]}

`GridSearchCV` will perform separate cross-validation for each of these dictionaries *and* the combinations within them.

Let me break it down further:

**For the first dictionary (PCA and NMF):**

1.  `GridSearchCV` will perform a 5-fold cross-validation (assuming the default `cv=5`).
2.  For **each fold**, it will:
    *   Try `PCA` with `n_components` set to 2, then 4, then 8, and for each of those, it will try all the `C` values in `C_OPTIONS`.
    *   Try `NMF` with `n_components` set to 2, then 4, then 8, and for each of those, it will try all the `C` values in `C_OPTIONS`.
    *   **This part is doing PCA and NMF on the same folds.** It tests PCA, then it tests NMF *on the same fold*, in order to have a fair comparison between PCA and NMF.
3.  It will then average the performance scores (e.g., accuracy) across the 5 folds for each combination of `reduce_dim`, `n_components`, and `C`.

**For the second dictionary (SelectKBest):**

1.  `GridSearchCV` will perform another, **independent** 5-fold cross-validation.
2.  For **each fold**, it will:
    *   Try `SelectKBest` with `k` set to 2, then 4, then 8, and for each of those, it will try all the `C` values in `C_OPTIONS`.
3.  Again, it will average the performance scores across the 5 folds for each combination of `k` and `C`.

**In essence:**

*   **PCA and NMF are evaluated together using the same set of 5 folds in one set of cross-validation runs.**
*   **SelectKBest is evaluated in a separate set of 5-fold cross-validation runs.**

**Why this approach (continued)?**

*   **Fair Comparison:** By evaluating PCA and NMF on the same folds, `GridSearchCV` ensures a fair comparison between them. Any differences in performance are more likely due to the methods themselves rather than differences in the data used for training and testing in each fold.
*   **Independence:** Treating the `SelectKBest` dictionary separately makes sense because it uses a fundamentally different feature selection approach compared to PCA and NMF. There's no need for a direct fold-by-fold comparison with PCA/NMF.

**Key takeaway:** Each dictionary in `param_grid` triggers its own independent cross-validation process, and within each dictionary, different hyperparameter combinations are compared using the same folds to ensure fairness.

# Follow-up Question 3: Combining Everything into a Single Dictionary in param_grid


User: Thanks for explaining, that makes sense! But if one wanted to, could the param_grid variable just have been one dictionary like so? And it would be essentially the same thing, other than now SelectKBest was also tested in the same 5 folds?

In [49]:
param_grid = [
    {
        "reduce_dim": [PCA(iterated_power=7), NMF(max_iter=1_000), SelectKBest(mutual_info_classif)],
        "reduce_dim__n_components": N_FEATURES_OPTIONS,
        "classify__C": C_OPTIONS,
    }
]

You're almost right, but there's a subtle but important difference in how `GridSearchCV` would interpret that single dictionary `param_grid` and how `SelectKBest` works with hyperparameters.

Let's break it down:

**What would happen with your proposed `param_grid`:**

In [50]:
param_grid = [
    {
        "reduce_dim": [PCA(iterated_power=7), NMF(max_iter=1_000), SelectKBest(mutual_info_classif)],
        "reduce_dim__n_components": N_FEATURES_OPTIONS,
        "classify__C": C_OPTIONS,
    }
]

`GridSearchCV` would still treat each combination as a separate experiment. It would iterate through all combinations of:

*   `reduce_dim`: PCA, NMF, and then SelectKBest
*   `reduce_dim__n_components`: 2, 4, and 8
*   `classify__C`: 1, 10, 100, 1000

**The problem is with `reduce_dim__n_components` and `SelectKBest`.**

*   PCA and NMF use the `n_components` hyperparameter to determine the number of components to extract.
*   `SelectKBest` uses a hyperparameter named `k` to determine the number of top features to select. **It does not have an `n_components` hyperparameter.**

**Consequences:**

When `GridSearchCV` gets to `SelectKBest`, it will try to set `reduce_dim__n_components` for `SelectKBest`, but `SelectKBest` will not understand what to do with `n_components`. This will likely raise an error or cause unexpected behavior because you're trying to apply a hyperparameter that doesn't exist for that particular method.

It would **not** do what we want, which is to have `"reduce_dim__k"` apply to `SelectKBest`, and have `"reduce_dim__n_components"` apply to PCA and NMF.
